In [8]:
import torch

from systems import SO, LienardSigmoid, LienardPoly, Selkov, BZreaction, VanDerPol
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [3]:
N = 1000 # number of observed vectors
T_max = 3 # maximum simulation time
n_systems = 200 # number of systems to create
dim = 2 # dimension of data
noise = 0  # standard deviation of the noise added to the data
root = 'data/'

# define systems of which data should be generated
systems = [
    ['SO', SO],
    ['selkov', Selkov],
    ['bzreaction', BZreaction],
    ['lienard_poly', LienardPoly],
    ['lienard_sigmoid', LienardSigmoid],
    ['vanderpol', VanDerPol],
]

## Create and save data

In [ ]:
# iterate over systems defined above
for (name, system) in systems:
    # create save path for the specific system
    path = root + f'{name}/'
    Path(path).mkdir(exist_ok=True, parents=True)
    xs = []
    dxs = []
    print(name, flush=True)
    # iterate over the number of required systems
    for i in tqdm(range(n_systems)):
        # reproducability
        torch.manual_seed(i)
        np.random.seed(i)

        # sample a system with random parameters
        s = system()

        # generate random locations from simulated trajectories
        x = s.rand_on_traj(s.random_x(N=N, dim=dim), T=T_max)
        xs.append(x.numpy())

        # get velocities and add noise
        dx = s(0, x)
        dx = dx + noise*torch.randn_like(dx)
        dxs.append(dx.numpy())

    # the positions are saved as a numpy array with shape [n_systems, dim]
    xs = np.stack(xs)
    np.save(path+f'coords_T={T_max}.npy', xs)

    # the velocities are saved as a numpy array with shape [n_systems, dim]
    dxs = np.stack(dxs)
    np.save(path+f'X_T={T_max}.npy', dxs)